<a href="https://colab.research.google.com/github/revanthmadasu/LifestyleWellbeing/blob/data-preprocessing/LifestyleWellbeing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Importing the data

In [2]:
dataset = pd.read_csv('/content/data/Wellbeing_and_lifestyle_data_Kaggle.csv')
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [8]:
print(X[0,:])

['7/7/15' 3 '2' 2 5 0 5 2 0 1 6 4 5 0 7 5 5 1 4 0 5 '36 to 50' 'Female']


In [6]:
print(X)

[['7/7/15' 3 '2' ... 5 '36 to 50' 'Female']
 ['7/7/15' 2 '3' ... 6 '36 to 50' 'Female']
 ['7/7/15' 2 '3' ... 3 '36 to 50' 'Female']
 ...
 ['3/14/21 8:35' 4 '3' ... 10 '21 to 35' 'Male']
 ['3/14/21 8:43' 1 '1' ... 8 '21 to 35' 'Female']
 ['3/14/21 9:03' 5 '4' ... 4 '21 to 35' 'Female']]


In [5]:
print(y)

[609.5 655.6 631.6 ... 716.6 682.  651.4]


Handling missing data

In [4]:
# from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
# imputer.fit(X[:, 1:3])
# X[:, 1:3] = imputer.transform(X[:, 1:3])

ValueError: ignored